In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [2]:
root = 'D:\\Database\\ashrae-energy-prediction-dataset\\'

In [3]:
#load the metadata
office=pd.read_csv(root+'office_building_meta0817.csv')
#load the origin metedata
origin_office=pd.read_csv(root+'office_building_meta.csv')

In [4]:
def add_timestamp(df):
    df["hour"] = df["timestamp"].dt.hour
#     df["day"] = df["timestamp"].dt.day
    df["weekend"] = df["timestamp"].dt.weekday
    df["month"] = df["timestamp"].dt.month
    df["dayofweek"] = df["timestamp"].dt.dayofweek

In [5]:
train_cleaned=pd.read_csv(root+'train_cleaned.csv')
train_cleaned["timestamp"] = pd.to_datetime(train_cleaned["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [9]:
def get_offwork(building_df):
    '''
    calculate the average energy consumption in workday nonworking hours and weekends for summer and winter.
    '''
    
    def month_summer(x):
        if (x>6 and x<9):
            return True
        else:
            return False
        
    def month_winter(x):
        if (x>11 or x<3):
            return True
        else:
            return False
    
    try:
        summer_month=building_df['month'].apply(month_summer)
    except KeyError:
        summer_month= building_df['month']==7
    else:
        summer_month= building_df['month']==7
        
    try:
        winter_month=building_df['month'].apply(month_winter)
    except KeyError:
        winter_month= building_df['month']==12
    else:
        winter_month= building_df['month']==12
        
    #workday_night_sm=building_df[building_df['weekend']<5][building_df['month'].isin(summer_month)]
    workday_night_sm=building_df[building_df['weekend']<5][building_df['month'].isin([7,8,9])]
    night_list=[]
    workday_list=[]
    for idx, day in workday_night_sm.groupby(workday_night_sm.timestamp.dt.date):
        day_mean=day['meter_reading'].dropna().mean()
        night_day=day[day['meter_reading']<=day_mean]
        workday_day=day[day['meter_reading']>day_mean]
        night_sm_day=night_day['meter_reading'].dropna().mean()
        workday_sm_day=workday_day['meter_reading'].dropna().mean()
        night_list.append(night_sm_day)
        workday_list.append(workday_sm_day)
        
    if len(night_list)>0:
        night_sm=sum(night_list)/len(night_list)
    else:
        night_sm=0
        
    if len(workday_list)>0:
        workday_sm=sum(workday_list)/len(workday_list)
    else:
        workday_sm=0.01
    
    weekend_list=[]
    weekend_sm=building_df[building_df['weekend']>=5][building_df['month'].isin([7,8,9])]
    for idx, day in weekend_sm.groupby(weekend_sm.timestamp.dt.date):
        day_mean=day['meter_reading'].mean()
        week_day=day[day['meter_reading']>day_mean]
        weekend_day_mean=week_day['meter_reading'].dropna().mean()
        weekend_list.append(weekend_day_mean)
    if len(weekend_list)>0:
        weekend_sm= sum(weekend_list)/len(weekend_list)
    else:
        weekend_sm=0
        
    #workday_night_wt=building_df[building_df['weekend']<5][building_df['month'].isin(winter_month)]
    workday_night_wt=building_df[building_df['weekend']<5][building_df['month'].isin([12,1,2])]
    night_wt_list=[]
    workday_wt_list=[]
    for idx, day in workday_night_wt.groupby(workday_night_wt.timestamp.dt.date):
        day_mean=day['meter_reading'].dropna().mean()
        night_day=day[day['meter_reading']<=day_mean]
        workday_day=day[day['meter_reading']>day_mean]
        
        night_wt_day=night_day['meter_reading'].dropna().mean()
        workday_wt_day=workday_day['meter_reading'].dropna().mean()
        
        night_wt_list.append(night_wt_day)
        workday_wt_list.append(workday_wt_day)
        
    if len(night_wt_list)>0:
        night_wt=sum(night_wt_list)/len(night_wt_list)
    else:
        night_wt=0
        
    if len(workday_wt_list)>0:
        workday_wt=sum(workday_wt_list)/len(workday_wt_list)
    else:
        workday_wt=0.01
    
    weekend_list=[]
    #weekend_wt=building_df[building_df['weekend']>=5][building_df['month'].isin(winter_month)]
    weekend_wt=building_df[building_df['weekend']>=5][building_df['month'].isin([12,1,2])]
    for idx, day in weekend_wt.groupby(weekend_wt.timestamp.dt.date):
        day_mean=day['meter_reading'].mean()
        weekend_day=day[day['meter_reading']>day_mean]
        weekend_day_mean=weekend_day['meter_reading'].dropna().mean()
        weekend_list.append(weekend_day_mean)
    if len(weekend_list)>0:
        weekend_wt= sum(weekend_list)/len(weekend_list)
    else:
        weekend_wt=0
    return round(night_sm,2), round(workday_sm,2),round(weekend_sm,2),round(night_wt,2),round(workday_wt,2),round(weekend_wt,2)

In [10]:
def get_season(bld_df):
    '''
    calculate the energy consumption in transition season for buildings in north hemisphere.
    '''
     #获取春天、过渡季、夏季的平均电量数据
    bld_sp_df=bld_df[bld_df['month']==4][bld_df['hour']>8][bld_df['hour']<20]
    bld_sm_df=bld_df[bld_df['month']==7][bld_df['hour']>8][bld_df['hour']<20]
    bld_at_df=bld_df[bld_df['month']==10][bld_df['hour']>8][bld_df['hour']<20]

    #春天平均
    elc_sp=bld_sp_df['meter_reading'].dropna().mean()
    elc_sm=bld_sm_df['meter_reading'].dropna().mean()
    elc_at=bld_at_df['meter_reading'].dropna().mean()
    
    TRS_sp=round(elc_sp/elc_sm,2)
    TRS_at=round(elc_at/elc_sm,2)
    return TRS_sp,TRS_at

In [11]:
def get_offwork_index(train_df_office,building_meta_df_office):
    '''
        calculate the EPI value for off workhous, and save into the meta-dataset
    '''
    NWH_sm_list=[]
    NWH_wt_list=[]
    WKD_sm_list=[]
    WKD_wt_list=[]
    for building_i in building_meta_df_office['building_id'].values:
        building_df=train_df_office[train_df_office['building_id']==building_i]
        preprocess(building_df)
        night_sm, workday_sm,weekend_sm,night_wt,workday_wt,weekend_wt= get_offwork(building_df)
        NWH_sm=night_sm/workday_sm
        NWH_wt=night_wt/workday_wt
        WKD_sm=weekend_sm/workday_sm
        WKD_wt=weekend_wt/workday_wt
        
        NWH_sm_list.append(round(NWH_sm,2))
        NWH_wt_list.append(round(NWH_wt,2))
        WKD_sm_list.append(round(WKD_sm,2))
        WKD_wt_list.append(round(WKD_wt,2))
        
    building_meta_df_office['NWH_sm']=NWH_sm_list
    building_meta_df_office['NWH_wt']=NWH_wt_list
    building_meta_df_office['WKD_sm']=WKD_sm_list
    building_meta_df_office['WKD_wt']=WKD_wt_list

In [12]:
def get_season_index(train_df_office,building_meta_df_office):
    '''
    calculate the EPI for transition season, and save into meta-dataset
    '''
    TRS_sp_list=[]
    TRS_at_list=[]
    for i in building_meta_df_office['building_id'].values:
        building_df=train_df_office[train_df_office['building_id']==i]
        preprocess(building_df)
        TRS_sp,TRS_at=get_season(building_df)
        TRS_sp_list.append(TRS_sp)
        TRS_at_list.append(TRS_at)
    building_meta_df_office['TRS_sp']=TRS_sp_list
    building_meta_df_office['TRS_at']=TRS_at_list

In [84]:
get_offwork_index(train_cleaned,origin_office)

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

building_i 9
103.51 146.98 110.23 72.12 166.42 66.71
building_i 15
215.81 324.55 259.92 206.29 316.55 257.46
building_i 17
66.46 109.69 85.31 114.3 147.16 154.86
building_i 19
195.1 257.03 208.47 146.59 182.73 168.09
building_i 21
69.72 89.47 80.01 41.03 61.23 49.6
building_i 25
338.23 582.31 294.2 245.82 390.49 206.07
building_i 26
87.58 205.57 164.61 95.88 214.47 205.5
building_i 28
273.22 325.45 284.81 178.62 227.55 197.52
building_i 32
215.9 422.65 274.25 201.87 461.38 320.62
building_i 38
431.97 454.13 436.51 569.66 605.37 610.49
building_i 39
307.21 397.78 315.58 295.93 361.94 310.63
building_i 41
179.43 207.31 196.02 215.01 244.09 247.02
building_i 53
0 0.01 0 0.0 nan nan
building_i 55
86.42 114.93 109.4 93.41 143.81 149.59
building_i 79
629.75 722.78 624.44 492.47 566.12 527.26
building_i 80
1466.5 2038.19 1923.29 1183.22 1553.69 1518.26
building_i 82
38.55 59.89 60.95 64.37 84.6 85.1
building_i 84
25.93 46.94 37.27 13.93 26.03 18.05
building_i 92
72.01 88.98 71.37 83.16 102.24

building_i 1127
110.71 151.03 123.25 105.21 149.95 128.44
building_i 1128
65.08 96.63 67.82 78.82 103.35 78.95
building_i 1129
102.93 155.28 120.3 103.08 161.2 117.39
building_i 1131
44.26 72.43 42.33 24.59 41.66 23.62
building_i 1133
117.78 163.31 117.91 116.79 164.34 119.06
building_i 1134
125.72 236.33 147.12 100.71 198.95 115.48
building_i 1136
40.91 101.45 36.31 43.56 86.51 39.6
building_i 1137
13.93 32.04 14.97 12.24 29.41 14.34
building_i 1139
1741.29 2524.67 1699.83 1028.2 1224.35 1081.92
building_i 1142
45.75 108.31 46.34 51.67 107.45 50.92
building_i 1143
66.53 127.78 76.64 62.48 106.83 59.68
building_i 1144
427.61 492.85 440.35 445.58 497.54 447.01
building_i 1148
916.33 1239.51 882.12 874.37 1202.85 852.55
building_i 1149
344.99 688.41 475.36 292.11 502.47 337.84
building_i 1150
138.74 200.89 158.35 78.92 214.0 142.89
building_i 1151
1091.64 1677.12 1331.27 234.64 291.49 255.25
building_i 1152
392.37 618.69 624.44 370.73 457.84 381.19
building_i 1159
1817.67 2468.58 1916.91

In [95]:
get_season_index(train_cleaned,origin_office)

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

Building 9 nan 1.15
Building 15 nan 1.07
Building 17 nan 1.26
Building 19 nan 0.87
Building 21 nan 1.02
Building 25 nan 0.86
Building 26 nan 1.1
Building 28 nan 0.9
Building 32 nan 1.27
Building 38 nan 1.27
Building 39 nan 1.03
Building 41 nan 1.13
Building 53 nan nan
Building 55 nan 1.19
Building 79 nan 0.9
Building 80 nan 0.83
Building 82 nan 1.1
Building 84 nan 0.68
Building 92 nan 1.1
Building 93 nan 1.12
Building 99 nan 0.76
Building 101 nan 0.99
Building 102 nan 0.49
Building 104 nan 1.09
Building 140 0.83 1.1
Building 141 0.87 1.2
Building 142 1.03 1.1
Building 143 0.75 1.06
Building 144 1.0 14.99
Building 145 0.9 1.19
Building 146 1.52 0.81
Building 147 1.07 1.09
Building 148 0.76 0.89
Building 149 0.99 0.93
Building 150 0.9 0.93
Building 151 0.66 0.96
Building 152 1.35 1.22
Building 153 0.91 1.05
Building 154 1.03 0.89
Building 155 1.09 1.02
Building 157 0.78 0.92
Building 158 0.57 0.66
Building 171 2.56 0.76
Building 172 1.25 1.12
Building 178 1.13 1.15
Building 188 0.86 0.82

In [97]:
origin_office.to_csv(root+'office_revised05173.csv')

In [13]:
from scipy import stats
def energy_compare_level(energy_series):
    '''
    report the energy level percentage comparing  with all other buildings.
    '''
    ranks_list=[]
    EUI_list=list(energy_series.values)
    sorted_dataDF_EUI=np.sort(EUI_list)
    for i in EUI_list:
        percentage=stats.percentileofscore(list(sorted_dataDF_EUI),i)
        ranks_list.append(format(percentage,'.2f'))
    ranked_series=pd.Series(ranks_list,index=energy_series.index)
    return ranked_series

In [14]:
building_meta_df_office=pd.read_csv(root+'office_revised05172.csv')

In [15]:
building_meta_df_office['NWH_sm_lv']=energy_compare_level(building_meta_df_office['NWH_sm'])
building_meta_df_office['NWH_wt_lv']=energy_compare_level(building_meta_df_office['NWH_wt'])
building_meta_df_office['WKD_sm_lv']=energy_compare_level(building_meta_df_office['WKD_sm'])
building_meta_df_office['WKD_wt_lv']=energy_compare_level(building_meta_df_office['WKD_wt'])
building_meta_df_office['TRS_sp_lv']=energy_compare_level(building_meta_df_office['TRS_sp'])
building_meta_df_office['TRS_at_lv']=energy_compare_level(building_meta_df_office['TRS_at'])

C:\Users\tzcha\Anaconda3\lib\site-packages\scipy\stats\stats.py:1823: RuntimeWarning:

invalid value encountered in less

C:\Users\tzcha\Anaconda3\lib\site-packages\scipy\stats\stats.py:1824: RuntimeWarning:

invalid value encountered in less_equal



In [18]:
building_meta_df_office=building_meta_df_office.dropna()

In [19]:
building_meta_df_office.to_csv(root+'office_building_meta0518.csv')

In [ ]:
building_meta_df_office['NWH_wt_lv']=energy_compare_level(building_meta_df_office['NWH_wt'])
building_meta_df_office['WKD_sm_lv']=energy_compare_level(building_meta_df_office['WKD_sm'])
building_meta_df_office['WKD_wt_lv']=energy_compare_level(building_meta_df_office['WKD_wt'])
building_meta_df_office['TRS_sp_lv']=energy_compare_level(building_meta_df_office['TRS_sp'])
building_meta_df_office['TRS_at_lv']=energy_compare_level(building_meta_df_office['TRS_at'])